# Classification

In [ ]:
import pandas as pd

data = pd.read_csv("train_data.tsv", delimiter='\t')
data.head()

,insert_chrom,insert_name,sequence,rna_dna_ratio,is_active
0,chr16,chr16:22572551-22572821,GTTCTCTATGCCATCTCACATTTATCCCCAAAGCCCAGGGAGGCTG...,1.179811,1
1,chr18,chr18:46761035-46761305,CAAAACATGGAGGTCTGAAGTCCCTTTTGTTCCTATCAGTATGGAG...,1.406439,1
2,chr1,chr1:209485218-209485488,TGTGGGAACAAGGAAGGAGTTAAGTTGGCATAAGCAGTTTAATAAG...,1.021255,0
3,chr3,chr3:45119427-45119697,TGCCCTATCTTGGATTCTGATTATGAGTCTCCGGGCAGCTGATGGC...,0.817915,0
4,chr14,chr14:51481635-51481905,GGTTTCCTTCGATCACCCAAGCTGGCTATTATTATCGTTCTTACAC...,0.756986,0


In [ ]:
!nvidia-smi

Fri Apr 18 17:38:30 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   55C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

https://huggingface.co/docs/transformers/en/model_doc/esm

other models to check: https://huggingface.co/facebook/esm2_t36_3B_UR50D

(na razie tylko dla klasyfikacji)

In [ ]:
import torch
from transformers import AutoTokenizer, EsmForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("facebook/esm2_t6_8M_UR50D")
model = EsmForSequenceClassification.from_pretrained("facebook/esm2_t6_8M_UR50D")
model2 = EsmForSequenceClassification.from_pretrained("facebook/esm2_t6_8M_UR50D")

for param in model2.parameters():
    param.requires_grad = False

# Unfreeze only the classifier (final layer) parameters
for param in model2.classifier.parameters():
    param.requires_grad = True


inputs = tokenizer(data["sequence"][0], return_tensors="pt")
print(inputs)
with torch.no_grad():
    logits = model(**inputs).logits

from sklearn.model_selection import train_test_split

device = 'cuda' if torch.cuda.is_available() else 'cpu'
optimizer = torch.optim.Adam(params=model.parameters(), lr=0.0001)
optimizer2 = torch.optim.Adam(params=model2.parameters(), lr=0.0001)

X = list(data["sequence"])
y = torch.Tensor([[0, 1] if dataPoint == 1 else [1, 0] for dataPoint in list(data["is_active"])])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=44)
X_train = tokenizer(X_train, return_tensors="pt")
X_test = tokenizer(X_test, return_tensors="pt")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/95.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/93.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/775 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/31.4M [00:00<?, ?B/s]

Some weights of EsmForSequenceClassification were not initialized from the model checkpoint at facebook/esm2_t6_8M_UR50D and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


{'input_ids': tensor([[ 0,  6, 11, 11, 23, 11, 23, 11,  5, 11,  6, 23, 23,  5, 11, 23, 11, 23,
          5, 23,  5, 11, 11, 11,  5, 11, 23, 23, 23, 23,  5,  5,  5,  6, 23, 23,
         23,  5,  6,  6,  6,  5,  6,  6, 23, 11,  6, 11,  6, 11,  5,  5, 11, 11,
         11,  6, 11, 11, 23,  5,  5,  6,  6, 11,  6,  5, 23,  5, 23,  5,  6, 23,
          5,  5,  6, 11,  5, 11,  6, 11,  6,  6, 23,  5,  6,  5,  6, 23,  5,  6,
          6,  6,  6, 23, 11, 23,  6,  5,  5, 11, 11, 23,  5,  6,  6, 23, 23, 11,
         23, 11,  6,  5, 11, 23, 11, 11, 11,  5,  5,  6,  6, 23, 23, 11,  6, 11,
          6, 11, 11, 11, 23, 23, 23, 23, 23, 11, 23, 23,  5, 23,  5, 11, 23,  5,
          6, 11,  6, 11, 11, 11, 23,  5,  6,  6,  5,  6,  6, 11,  6,  6,  5,  5,
          6,  5, 23, 11, 11,  6,  5,  5,  6, 23,  5, 23, 11,  6,  6,  6,  5,  5,
          6, 23, 11,  6, 11, 23, 11, 11,  6, 23,  5, 11, 11,  6, 23,  5, 11, 11,
          5,  5,  5, 23,  5,  5, 23,  5, 11, 11,  6, 23, 23,  5, 23,  5, 11,  5,
          6,  

In [ ]:
model = model.to(device)

# BCEWithLogitsLoss - sigmoid is already built-in!
loss_fn = torch.nn.BCEWithLogitsLoss()

X_train, y_train = X_train.to(device), y_train.to(device).float()
X_test, y_test = X_test.to(device), y_test.to(device).float()

train_losses, test_losses = [], []
acc_train, acc_test = [], []
epochs = 20
batch_size = 25

for epoch in range(epochs):
    ### TEST
    model.eval()
    correct_test = 0
    with torch.inference_mode():
        for i in range(0, len(X_test["input_ids"]), batch_size):
            input_ids = X_test["input_ids"][i:i+batch_size]
            attention_mask = X_test["attention_mask"][i:i+batch_size]
            test_batch = {"input_ids": input_ids, "attention_mask": attention_mask}
            y_logits_test = model(**test_batch).logits

            #y_pred_test = torch.Tensor([[1, 0] if pred[0] > pred[1] else [0, 1] for pred in y_logits_test]).to(device)
             y_pred_test = torch.nn.functional.one_hot(torch.argmax(y_logits_test, dim=1), num_classes=2).float()

            test_loss = loss_fn(y_pred_test, y_test[i:i+batch_size])
            test_losses.append(test_loss.cpu().detach().numpy())

            correct_test += torch.eq(y_pred_test, y_test[i:i+batch_size]).sum().item()

    acc_test.append(correct_test / (2*len(X_test["input_ids"])))
    print(epoch)

    ### TRAIN
    model.train()
    correct_train = 0
    for i in range(0, len(X_train["input_ids"]), batch_size):
        input_ids = X_train["input_ids"][i:i+batch_size]
        attention_mask = X_train["attention_mask"][i:i+batch_size]
        train_batch = {"input_ids": input_ids, "attention_mask": attention_mask}
        y_logits = model(**train_batch).logits

        y_pred = torch.Tensor([[1, 0] if pred[0] > pred[1] else [0, 1] for pred in y_logits]).to(device)

        loss = loss_fn(y_logits, y_train[i:i+batch_size])

        train_losses.append(loss.cpu().detach().numpy())

        correct_train += torch.eq(y_pred, y_train[i:i+batch_size]).sum().item()

        optimizer.zero_grad()

        loss.backward()

        optimizer.step()

    acc_train.append(correct_train / (2*len(X_train["input_ids"])))
    if epoch % 1 == 0:
        print(f"Epoch: {epoch} | Loss: {loss:.5f}, Accuracy train: {acc_train[-1]:.4f} | Accuracy test: {acc_test[-1]:.4f} | Test loss: {test_loss:.5f}")


0
Epoch: 0 | Loss: 0.60563, Accuracy train: 0.6177 | Accuracy test: 0.6510 | Test loss: 0.62820
1
Epoch: 1 | Loss: 0.58067, Accuracy train: 0.6619 | Accuracy test: 0.6904 | Test loss: 0.65320
2
Epoch: 2 | Loss: 0.56417, Accuracy train: 0.6922 | Accuracy test: 0.7066 | Test loss: 0.60320
3
Epoch: 3 | Loss: 0.52990, Accuracy train: 0.7150 | Accuracy test: 0.7157 | Test loss: 0.62820
4
Epoch: 4 | Loss: 0.49725, Accuracy train: 0.7321 | Accuracy test: 0.6889 | Test loss: 0.62820
5
Epoch: 5 | Loss: 0.53633, Accuracy train: 0.7488 | Accuracy test: 0.6550 | Test loss: 0.67820
6
Epoch: 6 | Loss: 0.45464, Accuracy train: 0.7625 | Accuracy test: 0.6819 | Test loss: 0.67820
7
Epoch: 7 | Loss: 0.39616, Accuracy train: 0.7841 | Accuracy test: 0.6998 | Test loss: 0.62820
8
Epoch: 8 | Loss: 0.34925, Accuracy train: 0.8107 | Accuracy test: 0.7002 | Test loss: 0.70320
9
Epoch: 9 | Loss: 0.32448, Accuracy train: 0.8419 | Accuracy test: 0.6844 | Test loss: 0.70320
10
Epoch: 10 | Loss: 0.15171, Accuracy t

KeyboardInterrupt: 

In [ ]:
model2 = model.to(device)

# BCEWithLogitsLoss - sigmoid is already built-in!
loss_fn = torch.nn.BCEWithLogitsLoss()

train_losses, test_losses = [], []
acc_train, acc_test = [], []
epochs = 20
batch_size = 25

for epoch in range(epochs):
    ### TEST
    model2.eval()
    correct_test = 0
    with torch.inference_mode():
        for i in range(0, len(X_test["input_ids"]), batch_size):
            input_ids = X_test["input_ids"][i:i+batch_size]
            attention_mask = X_test["attention_mask"][i:i+batch_size]
            test_batch = {"input_ids": input_ids, "attention_mask": attention_mask}
            y_logits_test = model2(**test_batch).logits

            #y_pred_test = torch.Tensor([[1, 0] if pred[0] > pred[1] else [0, 1] for pred in y_logits_test]).to(device)
             y_pred_test = torch.nn.functional.one_hot(torch.argmax(y_logits_test, dim=1), num_classes=2).float()

            test_loss = loss_fn(y_pred_test, y_test[i:i+batch_size])
            test_losses.append(test_loss.cpu().detach().numpy())

            correct_test += torch.eq(y_pred_test, y_test[i:i+batch_size]).sum().item()

    acc_test.append(correct_test / (2*len(X_test["input_ids"])))
    print(epoch)

    ### TRAIN
    model2.train()
    correct_train = 0
    for i in range(0, len(X_train["input_ids"]), batch_size):
        input_ids = X_train["input_ids"][i:i+batch_size]
        attention_mask = X_train["attention_mask"][i:i+batch_size]
        train_batch = {"input_ids": input_ids, "attention_mask": attention_mask}
        y_logits = model2(**train_batch).logits

        y_pred = torch.Tensor([[1, 0] if pred[0] > pred[1] else [0, 1] for pred in y_logits]).to(device)

        loss = loss_fn(y_logits, y_train[i:i+batch_size])

        train_losses.append(loss.cpu().detach().numpy())

        correct_train += torch.eq(y_pred, y_train[i:i+batch_size]).sum().item()

        optimizer2.zero_grad()

        loss.backward()

        optimizer2.step()

    acc_train.append(correct_train / (2*len(X_train["input_ids"])))
    if epoch % 1 == 0:
        print(f"Epoch: {epoch} | Loss: {loss:.5f}, Accuracy train: {acc_train[-1]:.4f} | Accuracy test: {acc_test[-1]:.4f} | Test loss: {test_loss:.5f}")


Zatrzymałem po około 2h na GPU Colabowym, 20 epok, celność ~0.69 na zbiorze testowym

In [ ]:
# final eval
model.eval()
model2.eval()
correct_test = 0
correct_test2 = 0
with torch.inference_mode():
    for i in range(0, len(X_test["input_ids"]), batch_size):
        input_ids = X_test["input_ids"][i:i+batch_size]
        attention_mask = X_test["attention_mask"][i:i+batch_size]
        test_batch = {"input_ids": input_ids, "attention_mask": attention_mask}
        y_logits_test = model(**test_batch).logits
        y_logits_test2 = model2(**test_batch).logits

        #y_pred_test = torch.Tensor([[1, 0] if pred[0] > pred[1] else [0, 1] for pred in y_logits_test]).to(device)
        y_pred_test = torch.nn.functional.one_hot(torch.argmax(y_logits_test, dim=1), num_classes=2).float()
        y_pred_test2 = torch.nn.functional.one_hot(torch.argmax(y_logits_test2, dim=1), num_classes=2).float()

        test_loss = loss_fn(y_pred_test, y_test[i:i+batch_size])
        test_loss2 = loss_fn(y_pred_test2, y_test[i:i+batch_size])
        correct_test += torch.eq(y_pred_test, y_test[i:i+batch_size]).sum().item()
        correct_test2 += torch.eq(y_pred_test2, y_test[i:i+batch_size]).sum().item()

print("Model - Final accuracy:", correct_test / (2*len(X_test["input_ids"])))
print("Model (frozen) - Final accuracy:", correct_test2 / (2*len(X_test["input_ids"])))

In [ ]:
model.save_pretrained("./model_trained", from_pt=True)
model2.save_pretrained("./model_trained_frozen", from_pt=True)

# Regression

In [ ]:
import pandas as pd

data = pd.read_csv("train_data.tsv", delimiter='\t')
data.head()

In [ ]:
import torch
from transformers import AutoTokenizer, EsmForSequenceClassification

model_name = "facebook/esm2_t6_8M_UR50D"

from transformers import EsmModel
import torch.nn as nn

class EsmForRegression(nn.Module):
    def __init__(self, model_name):
        super().__init__()
        self.backbone = EsmModel.from_pretrained(model_name)
        hidden_size = self.backbone.config.hidden_size
        self.regressor = nn.Linear(hidden_size, 1)

    def forward(self, input_ids, attention_mask=None):
        outputs = self.backbone(input_ids=input_ids, attention_mask=attention_mask)
        pooled = outputs.last_hidden_state[:, 0]  # Use [CLS] token embedding
        return self.regressor(pooled)

class EsmForRegressionFrozen(nn.Module):
    def __init__(self, model_name):
        super().__init__()
        self.backbone = EsmModel.from_pretrained(model_name)
        for param in self.backbone.parameters():
            param.requires_grad = False
        hidden_size = self.backbone.config.hidden_size
        self.regressor = nn.Linear(hidden_size, 1)

    def forward(self, input_ids, attention_mask=None):
        outputs = self.backbone(input_ids=input_ids, attention_mask=attention_mask)
        pooled = outputs.last_hidden_state[:, 0]  # Use [CLS] token embedding
        return self.regressor(pooled)


tokenizer = AutoTokenizer.from_pretrained(model_name)
model = EsmForRegression(model_name)
model2 = EsmForRegressionFrozen(model_name)

inputs = tokenizer(data["sequence"][0], return_tensors="pt")
print(inputs)
with torch.no_grad():
    logits = model(**inputs)
    print(logits)
    print(data["rna_dna_ratio"][0])



In [ ]:
X_train = {key: val.to(device) for key, val in X_train.items()}
X_test = {key: val.to(device) for key, val in X_test.items()}

In [ ]:
from sklearn.model_selection import train_test_split

device = 'cuda' if torch.cuda.is_available() else 'cpu'
optimizer = torch.optim.Adam(params=model.parameters(), lr=0.0001)
optimizer2 = torch.optim.Adam(params=model2.parameters(), lr=0.0001)

X = list(data["sequence"])
y = list(data["rna_dna_ratio"])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=44)

In [ ]:
from torch.utils.data import Dataset, DataLoader

class SequenceRegressionDataset(Dataset):
    def __init__(self, sequences, targets, tokenizer):
        self.encodings = tokenizer(sequences, padding=True, truncation=True, return_tensors="pt")
        self.targets = torch.tensor(targets, dtype=torch.float).unsqueeze(1)

    def __len__(self):
        return len(self.targets)

    def __getitem__(self, idx):
        item = {key: val[idx].to(device) for key, val in self.encodings.items()}
        item["labels"] = self.targets[idx].to(device)
        return item

train_dataset = SequenceRegressionDataset(X_train, y_train, tokenizer)
test_dataset = SequenceRegressionDataset(X_test, y_test, tokenizer)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32)

In [ ]:
model = model.to(device)
epochs = 20
loss_fn = nn.MSELoss()
for epoch in range(epochs):
    model.train()
    total_loss = 0
    for batch in train_loader:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        loss = loss_fn(outputs, labels)

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        total_loss += loss.item()

    model.eval()
    total_loss = 0
    for batch in dataloader:
        input_ids = batch["input_ids"]
        attention_mask = batch["attention_mask"]
        labels = batch["labels"]

        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        loss = loss_fn(outputs, labels)

        total_loss += loss.item()

    avg_loss = total_loss / len(dataloader)
    print(f"Avg MSE Loss = {avg_loss:.4f}")

In [ ]:
torch.save(model.state_dict(), "esm_regression_model.pt")

In [ ]:
model2 = model2.to(device)
epochs = 20
loss_fn = nn.MSELoss()
for epoch in range(epochs):
    model2.train()
    total_loss = 0
    for batch in train_loader:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        outputs = model2(input_ids=input_ids, attention_mask=attention_mask)
        loss = loss_fn(outputs, labels)

        loss.backward()
        optimizer2.step()
        optimizer2.zero_grad()

        total_loss += loss.item()

    avg_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch+1}: Avg Train Loss = {avg_loss:.4f}")

In [ ]:
def eval_reg(model, dataloader):
  loss_fn = nn.MSELoss()
  model.eval()
  total_loss = 0
  for batch in dataloader:
      input_ids = batch["input_ids"]
      attention_mask = batch["attention_mask"]
      labels = batch["labels"]

      outputs = model(input_ids=input_ids, attention_mask=attention_mask)
      loss = loss_fn(outputs, labels)

      total_loss += loss.item()

  avg_loss = total_loss / len(dataloader)
  print(f"Avg MSE Loss = {avg_loss:.4f}")

In [ ]:
print("Model:")
eval_reg(model, test_dataloader)
print("Model (frozen):")
eval_reg(model2, test_dataloader)